In [96]:
import os
import json
import random
import string
from typing import Annotated, TypedDict, List, Sequence
from langchain_groq import ChatGroq 
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.types import Command
from langchain_core.messages import BaseMessage,HumanMessage,SystemMessage,AIMessage
from langgraph.graph.message import add_messages
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from dotenv import load_dotenv
from langgraph.prebuilt import InjectedState

In [2]:
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

In [3]:
llm = ChatGroq(model="llama-3.1-8b-instant")

In [156]:
class SupervisorState(MessagesState):
    """
        State for multi-agent system
    """   
    next_agent: str
    user_input: str
    user_intent: str
    complaint: List[dict]
    question:  List[dict]

# Supervisor Node

def supervisor_node(state:SupervisorState)->SupervisorState:
    system_prompt = """
        You are a supervisor. Based on user query decide the intent.
        Options: inquiry, complaint, retention. Just return the intent.

        - "My laptop stopped working" -> complaint
        - "Show me more info on my TCL TV" -> inquiry
        - "I want to cancel my Netflix subscription" -> retention
        - "My smartphone battery is not charging" -> complaint
        - "I want to know the specs of my AirPods" -> inquiry
        - "I want to cancel my gym membership" -> retention
        """
    
    response = llm.invoke([
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":state["user_input"]}
        ])
    
    intent=response.content.strip().lower()
    print("Intent is:",intent)
    if intent=="inquiry":
        return Command(goto="Inquiry Agent")
    elif intent=="complaint":
        return Command(goto="Complaint Agent")
    elif intent=="retention":
        return Command(goto="Retention Agent")
    else:
        return Command(goto="Fallback")


In [157]:
# Worker nodes

def inquiry_node(state:SupervisorState)->SupervisorState:
    return {'messages':[f"Inquiry Handled: {state['user_input']}"]}

def retention_node(state:SupervisorState)->SupervisorState:
    return {'messages':[f"Retention Handled: {state['user_input']}"]}

def fallback(state:SupervisorState)->SupervisorState:
    return {'messages':[f"Sorry, I couldn't understand your message"]}

In [158]:
@tool
def nlu(user_input:str)->dict:
    """It extracts the entities from the user input"""

    system_prompt="""
    Extract the following from the customer message:
    - product
    - issue_type
    - purchase_date

    Respond ONLY with a valid JSON object in this exact format:
    {
        "product": string or null,
        "issue_type": string or null,
        "purchase_date": string or null
    }"""
    response = llm.invoke([
        {'role':'system','content':system_prompt},
        {'role':'user','content':user_input}
    ])
    complaint = json.loads(response.content) 
    return {'complaint':complaint}

@tool
def ask_missing_info(missing_values:List[str])->dict:
    """
    It will ask the missing information to user to get the 
    remaining details missing from the user input
    """

    system_prompt=f"""You are a helpful assistant.
        Ask the user for the missing information in a clear and concise way, 
        one piece at a time, so we can complete the complaint record.
        Missing values are: {missing_values}
        """
    response = llm.invoke([{'role':'system','content':system_prompt}])
    return {'question':response.content}

@tool
def create_ticket(complaint: dict)->dict:
    """
    Tool to create a complaint ticket.
    Generates a ticket ID like IG408C90.
    """
    prefix = ''.join(random.choices(string.ascii_uppercase,k=2))
    number1 = random.randint(100,1000)
    mid = ''.join(random.choices(string.ascii_uppercase))
    number2 = random.randint(10,100)
    ticket_id = f"{prefix}{number1}{mid}{number2}"

    ticket={
        "ticket_id":ticket_id,
        "status":"created",
        "details":complaint
    }
    
    return ticket
    

In [159]:
def complaint_node(state:SupervisorState)->SupervisorState:
   """
      This node handles complaint and helps to resolve complaint.  
   """
   # system_prompt = """
   # You are an assistant. Use nlu tool to extract product, issue_type, purchase_date
   # from customer messages whenever needed.
   # """ 
   # llm_tool = llm.bind_tools([nlu])
   # response = llm_tool.invoke([{'role':'system','content':system_prompt},
   #                            {'role':'user','content':state['user_input']}])

   
   extracted=nlu(state['user_input'])
   print(extracted)
   missing_values=[k for k,v in extracted['complaint'].items() if v is None]
   print("Missing Values", missing_values)

   question=ask_missing_info({'missing_values':missing_values})
   print(question)
   state['question']=question

   print("creating ticket")
   ticket=create_ticket(extracted)
   print(ticket)

   return state

In [160]:
graph = StateGraph(SupervisorState)
graph.add_node("Supervisor",supervisor_node)
graph.add_node("Inquiry Agent",inquiry_node)
graph.add_node("Complaint Agent",complaint_node)
graph.add_node("Retention Agent",retention_node)
graph.add_node("Fallback", fallback)

graph.add_edge(START, "Supervisor")

app = graph.compile()

In [162]:
response = app.invoke({"user_input":"I want to know about iphone17pro"})

2025-09-29 23:32:54,389 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Intent is: inquiry
